In [1]:
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import os

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [26]:
global_connections_dict = {x:[] for x in ['before', 'after']}
nodes = dict()
for filename in os.listdir('output/'): 
    if 'connection_strength' in str(filename):
        with open('output/' + filename, 'rb') as file:
            connections = pickle.load(file)
        try: 
            connections_dict = list(connections.values())[0]
            
            [global_connections_dict['before'].append(edge) for edge in list(connections_dict['before'].values())]
            [global_connections_dict['after'].append(edge) for edge in list(connections_dict['after'].values())]
            
        except:
            pass
    elif 'nodes' in str(filename):
        with open('output/' + filename, 'rb') as file:
            channel = pickle.load(file)
            try:
                nodes.update(channel)
            except:
                pass

In [27]:
G = nx.Graph()
edge_list = global_connections_dict['after']
for edge in edge_list:
    if edge['strength'] > 1:
        G.add_edge(edge['origin_id'], 
                edge['destination_id'], 
                origin_title=edge['origin_title'],
                destination_title=edge['destination_title'],
                strength=edge['strength'],
                type=edge['type']
        )

node_labels = list(G.nodes())
titles = dict()
usernames = dict()
for node in node_labels:
    titles[node] = nodes[node].title
    usernames[node] = nodes[node].username
nx.set_node_attributes(G, titles, 'label')
nx.set_node_attributes(G, titles, 'username')

node_color = {}
node_radius = {}
edge_width = {}
FORWARD_COLOR, MENTION_COLOR = "darkgrey", "red"
for start_node, end_node, edge_attr in G.edges(data=True):
    node_color[end_node] = FORWARD_COLOR if edge_attr["type"] == 0 else MENTION_COLOR
    edge_width[(start_node, end_node)] = 2/(1+np.exp(-2*(edge_attr["strength"]-2)))
    node_radius[end_node] = 5/(1+np.exp(-2*(edge_attr["strength"]-2)))

nx.set_node_attributes(G, node_color, 'node_color')
nx.set_node_attributes(G, node_radius, 'node_radius')
nx.set_edge_attributes(G, edge_width, 'edge_width')


In [28]:
title = 'Загальний граф'
plot = figure(
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title
)

plot.sizing_mode = 'scale_width'
plot.grid.grid_line_color = None

network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0), weight='strength')
network_graph.node_renderer.glyph = Circle(size='node_radius', fill_color='node_color')
network_graph.edge_renderer.glyph = MultiLine(line_color="darkgrey", line_alpha=1, line_width='edge_width')

x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = nx.get_node_attributes(G, 'label')
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in network_graph.layout_provider.graph_layout]})
titles = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='8px', background_fill_alpha=.7)


plot.renderers.append(network_graph)
plot.renderers.append(titles)


show(plot)